In [1]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

filepath_in = 'data/sample_data_for_task1.csv'
filepath_out = 'data/processed_data.csv'

### Loading and inspecting the dataset

In [2]:
df = pd.read_csv(filepath_in)
df.head()

,text,label
0,zucker fabrik,ft
1,Lebensmittel kommssionierung,ft
2,geländer biegen,mr
3,gebäudeausrüstung technische,ct
4,kürbiskernöl softgels,ft


In [3]:
df.shape

(37295, 2)

In [4]:
# Our target variable will be 'label'
# Let's have a look at the distribution of its values
df['label'].value_counts()

label
ft     11226
pkg     9617
ct      5061
mr      5016
ch      3688
cnc     2587
Name: count, dtype: int64

### Vectorize the texts

In [5]:
tokenizer = CountVectorizer().build_tokenizer()
tokenized_text = df['text'].apply(tokenizer)

In [6]:
embedding_dim = 100  # Dimensionality of the word vectors
window_size = 5  # Maximum distance between the current and predicted word within a sentence
min_count = 1  # Ignores all words with a total frequency lower than this
model = Word2Vec(sentences=tokenized_text, vector_size=embedding_dim, window=window_size, min_count=min_count)

In [7]:
def text_to_embeddings(text):
    tokens = tokenizer(text)
    embeddings = []
    for token in tokens:
        if token in model.wv:
            embeddings.append(model.wv[token])
    if embeddings:
        return np.mean(embeddings, axis=0)  # Average of word embeddings in the text
    else:
        return np.zeros(embedding_dim)  # If no embeddings found, return zero vectors


In [8]:
df['embeddings'] = df['text'].apply(text_to_embeddings)
col_list = [f'emb_{i}' for i in range(100)]
df[col_list] = pd.DataFrame(df['embeddings'].tolist(), index= df.index)
df.drop(columns=['text', 'embeddings'], inplace=True)

/var/folders/vh/836sq5cd5h1_6zld30lb7gvc0000gn/T/ipykernel_35242/1631475004.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_list] = pd.DataFrame(df['embeddings'].tolist(), index= df.index)
/var/folders/vh/836sq5cd5h1_6zld30lb7gvc0000gn/T/ipykernel_35242/1631475004.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_list] = pd.DataFrame(df['embeddings'].tolist(), index= df.index)


In [9]:
df.to_csv(filepath_out)